In [1]:
import codes.trijunction as trijunction
import json
import importlib
import codes.optimization as optimization
import codes.parameters as parameters
import tinyarray as ta
import numpy as np

VoronoiMesh version 0.1
DiscretePoisson version 0.3
Shape version: 0.1
SystEquations version: 0.3
Solver version 0.1


In [19]:
from scipy.optimize import minimize, minimize_scalar

In [2]:
with open('/home/tinkerer/trijunction-design/codes/config.json', 'r') as f:
    config = json.load(f)

In [3]:
system = trijunction.Trijunction(config)

In [4]:
system.make_system()

Finding closed voronoi cells
Done selecting closed voronoi cells
Calculating points distance
Done
Calculating ridges hypersurfaces
Done
Finding  hypervolume for closed voronoi cells
Done finding hypervolume for closed voronoi cells
Done calculating surface
Done calculating distance
Done calculating capacitance matrix


100%|██████████| 13/13 [00:37<00:00,  2.91s/it]


In [5]:
pair = 'left-right'
voltages = parameters.pair_voltages()

In [6]:
zero_potential = dict(
    zip(
        ta.array(system.site_coords[:, [0, 1]] - system.offset),
        np.zeros(len(system.site_coords)),
    )
)

In [7]:
params = parameters.junction_parameters()
params.update(voltages[pair])
params.update(potential=zero_potential)
kwant_args = list(system.optimiser_arguments().values())

In [14]:
x = [-3.0e-3, -3.0e-3, -7.0e-3, 3e-3]
voltages = parameters.voltage_dict(x)

In [15]:
params.update(voltages)

In [17]:
opt_args = tuple([pair, params, kwant_args])

In [18]:
# loss to optimize voltages
# optimization.loss([0.0,1,1,1], *opt_args)

# loss to optimize phases
optimization.loss(0.1, *opt_args)

15.776435153682042

In [24]:
%%time

sol = minimize_scalar(
    optimization.loss,
    0.2, 
    args=opt_args, 
    bounds=(0, 2), 
    method="bounded"
)

CPU times: user 1min 54s, sys: 26.8 s, total: 2min 21s
Wall time: 2min 20s


In [26]:
optimal_phase = parameters.phase_pairs(opt_args[0], 
                                              sol.x * np.pi)

In [27]:
optimal_phase

{'phi1': 0.9985266761097111, 'phi2': 0}